### RatU Day2 NSD recording info
- No timestamps were deleted after concatenating .dat files from various folders of open-ephys

In [28]:
import numpy as np
import subjects
import matplotlib.pyplot as plt

sess = subjects.nsd.ratUday2[0]
print(sess.recinfo)

filename: /data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.xml 
# channels: 192
sampling rate: 30000
lfp Srate (downsampled): 1250



### Set probe configuration
- RatU_Day2NSD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
%matplotlib widget
from neuropy.core import Shank,Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank =Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0,-15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

#--- dignostic biochip 4 shanks (4 shanks were bad) ----------- 
shanks = []
for i in range(8,12):

    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=30,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = Probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp =ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

# prbgrp.filename = sess.fileprefix.with_suffix(".probegroup.npy")
# prbgrp.save(sess.filePrefix.with_suffix('.probegroup.npy'))
plot_probe(prbgrp)

## Writing to json format for spyking-circus 

In [ ]:
from neuropy.utils import probe_util

file = sess.filePrefix.with_suffix('.prb')
probe_util.write_spyking_circus(file,sess.probegroup,shanksCombine=True)


# Create experimental paradigm
- pre sleep is a little shorter
- animal was lazy on re-maze

In [ ]:
sess.paradigm.to_dataframe()

In [ ]:
import pandas as pd
from neuropy.core import Epoch

datetime_data = pd.read_csv(sess.filePrefix.with_suffix('.datetime.csv'))
durations = datetime_data.nFrames/sess.recinfo.dat_sampling_rate
epochs = pd.DataFrame(
    {
        "start": [0,9545,12856,45317],
        "stop": [9544,12855,45316,48496],
        "label": ["pre", "maze", "post",'re-maze'],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.filename = sess.filePrefix.with_suffix(".paradigm.npy")
# paradigm.save()


In [ ]:
durations

In [ ]:
np.cumsum(durations)

# Detect epochs
Here we will various types of epochs which typical for hippocampal recordings.

## Artifacts epochs
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
%matplotlib widget
from neuropy.analyses import detect_artifact_epochs
from neuropy import plotting
from neuropy.utils import signal_process
from neuropy.core import Signal
print(sess)
signal = sess.eegfile.get_signal([23,155])
# filt_trace = signal_process.filter_sig.highpass(signal.traces,cutoff=400)
# filtered_signal = Signal(np.median(filt_trace,axis=0).reshape((1,-1)),signal.sampling_rate)
artifact_epochs = detect_artifact_epochs(signal, thresh=10)
artifact_epochs.filename = sess.filePrefix.with_suffix(".artifact.npy")
artifact_epochs.save()
# plotting.plot_artifact_epochs(artifact_epochs,sess.eegfile.get_signal([175]),downsample_factor=5)


In [ ]:
sess.artifact.to_dataframe()

### write artifact to neuroscope

In [ ]:

sess.recinfo.write_epochs(sess.artifact,ext='.art')

### write artifact epochs to spyking circus format

In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix('.dead')
SpykingCircusIO.write_epochs(file,sess.artifact)

## Sleep scoring

In [ ]:
from neuropy.core import Signal
from neuropy.analyses import brainstates

signal = sess.eegfile.get_signal() 
brainstates = brainstates.detect_brainstates_epochs(signal=signal,probe=sess.probegroup)

In [ ]:
brainstates.filename = sess.filePrefix.with_suffix('.brainstates')
brainstates.save()

## Ripple epochs
To detect ripples one also needs probegroup.

In [ ]:
from neuropy.analyses import oscillations
signal = sess.eegfile.get_signal()
ripple_epochs =oscillations.detect_ripple_epochs(signal, sess.probegroup)
ripple_epochs.filename = sess.filePrefix.with_suffix('.ripple.npy')
ripple_epochs.save()

In [ ]:
signal = sess.eegfile.get_signal(channel_id=[1, 2, 3, 4], t_start=1, t_stop=1.2)
plotting.plot_signal_traces(signal)

# Neurons

## Importing spiketrains from Phy

In [ ]:
from neuropy.io import PhyIO
from neuropy.core import Neurons
from pathlib import Path
import numpy as np

cluster_path = Path("/home/bapung/Documents/ClusteringHub/spykcirc/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38-1.GUI")
chan_grps = sess.recinfo.channel_groups
phy_data = PhyIO(cluster_path)
spiketrains =phy_data.spiketrains
peak_chans = phy_data.peak_channels
waveforms = phy_data.waveforms
shank_id = sess.probegroup.get_shank_id_for_channels(peak_chans)

neuron_type_id = phy_data.cluster_info.q.values
neuron_type = np.ones(len(neuron_type_id), dtype="U5")
neuron_type[neuron_type_id<4] = 'pyr'
neuron_type[neuron_type_id==6] = 'mua'
neuron_type[neuron_type_id==8] = 'inter'


neurons = Neurons(
    np.array(spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=phy_data.sampling_rate,
    peak_channels=peak_chans,
    waveforms=np.array(waveforms,dtype='object'),
    shank_ids=np.array(shank_id).astype(int),
    neuron_type=neuron_type,
    metadata={'cluster_path':cluster_path}
)

neurons.filename = sess.filePrefix.with_suffix('.neurons')
neurons.save()


In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from neuropy.plotting import plot_raster


plt.plot(phy_data.peak_waveforms[0])
plot_raster(neurons,color='jet',add_vert_jitter=True)

## BinnedSpiketrain and Mua objects using Neurons

In [ ]:
mua =sess.neurons.get_mua()
mua.filename = sess.filePrefix.with_suffix(".mua.npy")
mua.save()   


In [ ]:
%matplotlib widget
from neuropy import plotting
smth_mua = sess.mua.get_smoothed(sigma=0.02)
plotting.plot_mua(smth_mua)

In [ ]:
from neuropy.analyses import detect_pbe_epochs

pbe = detect_pbe_epochs(smth_mua)
pbe.filename = sess.filePrefix.with_suffix('.pbe')
pbe.save()


# Position
- concatenated .dat file did not have any deleted timepoints

## Import position from optitrack

In [ ]:
from neuropy.io import OptitrackIO
from neuropy.core import Position
from pathlib import Path

opti_folder = sess.filePrefix.parent / 'position'
opti_data = OptitrackIO(dirname=opti_folder,scale_factor=0.25)

### Align position with .dat file
- t_error for maze position alignment were estimated using theta power and speed cross correlogram.

In [ ]:
import pandas as pd
from datetime import datetime

# ------- maze align corection ---------
t_error = [0, 0, 0, 0.334, 0, 0.619]
# ---- startimes of concatenated .dat files
tracking_sRate = opti_data.sampling_rate
rec_datetime = pd.read_csv(sess.filePrefix.with_suffix(".datetime.csv"))
data_time = []
for i, file_time in enumerate(rec_datetime["StartTime"]):
    # sync_time = rec_datetime['sync_nframes'][i]/rec_datetime['sync_rate'][i]
    tbegin = datetime.strptime(file_time, "%Y-%m-%d_%H-%M-%S") + pd.Timedelta(
        t_error[i], unit="sec"
    )
    # + pd.Timedelta(sync_time,unit='sec')
    nframes = rec_datetime["nFrames"][i]
    duration = pd.Timedelta(nframes / sess.recinfo.dat_sampling_rate, unit="sec")
    print(duration.total_seconds() * tracking_sRate)
    tend = tbegin + duration
    trange = pd.date_range(
        start=tbegin,
        end=tend,
        periods=int(duration.total_seconds() * tracking_sRate),
        # freq='16.66ms',
        closed="left",
    )
    data_time.extend(trange)
data_time = pd.to_datetime(data_time)

x, y, z = opti_data.get_position_at_datetimes(data_time)
traces = np.vstack((z, x, y))

position = Position(traces=traces, t_start=0, sampling_rate=60)
position.save(sess.filePrefix.with_suffix(".position.npy"))

## Linearize position

In [29]:
from neuropy.utils import position_util

for e in ['maze','re-maze']:
    maze = sess.paradigm[e].flatten()
    maze_pos = sess.position.time_slice(maze[0],maze[1])
    linear_pos = position_util.linearize_position(maze_pos)
    e = e.replace('-','')
    linear_pos.save(sess.filePrefix.with_suffix(f'.{e}.linear'))


/data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.maze.linear saved


/home/bapung/miniconda3/envs/data_analysis/lib/python3.10/site-packages/sklearn/manifold/_isomap.py:304: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/home/bapung/miniconda3/envs/data_analysis/lib/python3.10/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


/data/Clustering/sessions/RatU/RatUDay2NSD/RatU_Day2NSD_2021-07-24_08-16-38.remaze.linear saved


## Checking position alignment with .dat file
- Comparing theta power, speed and position to check if high theta periods are correlated with the speed of the animal

In [ ]:
%matplotlib widget
from neuropy.utils import signal_process
maze= sess.paradigm['maze'].flatten()
signal = sess.eegfile.get_signal([111],t_start=maze[0],t_stop=maze[1])
# spec = signal_process.SpectrogramBands(signal,window=1,overlap=0.8,norm_sig=True)

In [ ]:
from neuropy_viewer import time_frequency_viewer

time_frequency_viewer(signal,freq_params=(3,120,0.1))

In [ ]:
from neuropy.utils import signal_process
from scipy import stats

freqs = np.arange(1, 20, 0.5)
wavdec = signal_process.wavelet_decomp(
    stats.zscore(signal.traces[0]), sampfreq=1250, freqs=freqs
)

wvlt_power = wavdec.colgin2009()


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(wvlt_power,aspect='auto',cmap='jet',vmax=80000,extent=[signal.t_start,signal.t_stop,wavdec.freqs[0],wavdec.freqs[-1]],origin='lower')
plt.plot(pos_t,speed/10,'w')

In [ ]:
indx = np.where((wavdec.freqs>=8)&(wavdec.freqs<=16))[0]
theta_wvlt_power = np.median(wvlt_power[indx,:],axis=0)

In [ ]:
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
import numpy as np

position = sess.position.time_slice(maze[0],maze[1])
rpls = sess.ripple.time_slice(maze[0],maze[1]).starts
x = position.x[:-1]
y = position.y[:-1]
pos_t = position.time[:-1]
speed = gaussian_filter1d( position.speed,sigma =20)
theta = np.interp(pos_t,spec.time,spec.theta)
theta = gaussian_filter1d(theta,sigma=20)


plt.plot(pos_t,theta*100,'r')
# plt.plot(sess.position.time,sess.position.y)
plt.plot(signal.time,theta_wvlt_power/4000,'r')
plt.plot(rpls,100*np.ones_like(rpls),'*')
plt.plot(pos_t,speed,'g')
plt.plot(pos_t,x,'gray')
plt.plot(pos_t,y,'gray')
# plt.plot(pos_t,y,'k')
# plt.plot(sess.position.time[1:],np.diff(sess.position.x)*100)
# plt.xlim([1500,1600])
plt.ylim([-400,400])

In [ ]:
position.filename = sess.filePrefix.with_suffix('.maze.linear')
position.t_start = position.t_start +6
position.save()

In [ ]:
sess.paradigm['maze']/60

In [ ]:
%matplotlib widget
import scipy.signal as sg

xcorr = sg.correlate(theta,speed,mode='same')
lags= sg.correlation_lags(len(pos_t),len(pos_t),mode='same')

plt.plot(lags/position.sampling_rate,xcorr)
# plt.xlim([-20,20])




## Run direction

In [ ]:
from neuropy.utils import position_util
from neuropy.plotting import plot_epochs
from scipy.ndimage import gaussian_filter1d

run = position_util.run_direction(sess.lin_maze)
run.filename = sess.filePrefix.with_suffix('.running')
run.save()
_,ax = plt.subplots()
ax.plot(sess.lin_maze.time,sess.lin_maze.x)
# ax.plot(sess.lin_maze.time,gaussian_filter1d(sess.lin_maze.x,sigma=50))
plot_epochs(ax=ax,epochs=run,colors={'forward':'r','backward':'k'},alpha=0.2)


## Licking TTL epochs

In [ ]:
from neuropy.io import openephysio

ttl_folder = '/data/Clustering/sessions/RatU/RatUDay2NSD/TTL_1' 
ttl = openephysio.load_ttl_events(ttl_folder,sync_info=False)
print(ttl)

In [ ]:
file= '/data/Clustering/sessions/RatU/RatUDay2NSD/timestamps.npy'
data = np.load(file)
data[0]

In [ ]:
991423230/30000

In [ ]:
b = (ttl['timestamps']-985075968)/30000

In [ ]:
from neuropy.plotting import plot_epochs

_,ax = plt.subplots()
ax.plot(sess.position.time,sess.position.x)
# plot_epochs(ax=ax,epochs= sess.paradigm)

for val in b:
    ax.axvline(val+45321,color='k')

# ax.set_xlim(sess.paradigm['maze'])
